In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [2]:
scenarios = {"optimistic":"ssp1_2_6",
             "neutral":"ssp2_4_5",
             "pessimistic":"ssp5_8_5"}

In [3]:
scenario = scenarios["optimistic"]
df = pd.read_csv(f"../data/weather_agg_{scenario}.csv")

In [4]:
df.sort_values("year", ascending = True)

,scenario,department,code_dep,year,mean_temp,std_temp,mean_max_temp,std_max_temp,total_precip,std_precip,max_consec_rain_days,min_consec_rain_days,max_daily_precip,rainy_days_count,yield
0,historical,Ain,1,1982,282.77924,7.090540,287.48267,8.055373,0.027494,0.000120,10.0,1.0,0.000870,111.0,3.950078
2574,historical,Somme,80,1982,283.59534,6.709761,287.07242,7.748220,0.028498,0.000128,7.0,1.0,0.000754,111.0,6.037931
594,historical,Correze,19,1982,282.33575,6.604024,286.93512,7.655155,0.065636,0.000277,32.0,1.0,0.001701,163.0,2.363158
627,historical,Cote_d_Or,21,1982,282.69196,7.265679,287.25403,8.291865,0.043310,0.000184,20.0,1.0,0.001159,134.0,4.447979
2541,historical,Seine_et_Marne,77,1982,283.99490,7.501804,288.43646,8.690378,0.007367,0.000040,2.0,1.0,0.000351,25.0,5.697143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,ssp1_2_6,Eure_et_Loir,28,2050,285.22110,6.803133,289.15570,7.692380,0.036134,0.000164,10.0,1.0,0.001291,121.0,NaN
3188,ssp1_2_6,Ardennes,8,2050,284.22546,6.851576,288.13013,7.848469,0.024687,0.000107,7.0,1.0,0.000709,98.0,NaN
5708,ssp1_2_6,Seine_Maritime,76,2050,284.93118,6.034536,288.16920,6.794929,0.033093,0.000158,9.0,1.0,0.001694,114.0,NaN
4196,ssp1_2_6,Haute_Garonne,31,2050,286.33957,6.437269,291.23083,7.053396,0.032264,0.000166,7.0,1.0,0.001177,110.0,NaN


In [5]:
df.head()

,scenario,department,code_dep,year,mean_temp,std_temp,mean_max_temp,std_max_temp,total_precip,std_precip,max_consec_rain_days,min_consec_rain_days,max_daily_precip,rainy_days_count,yield
0,historical,Ain,1,1982,282.77924,7.090540,287.48267,8.055373,0.027494,0.000120,10.0,1.0,0.000870,111.0,3.950078
1,historical,Ain,1,1983,281.72400,7.357678,286.23380,8.096182,0.029012,0.000119,7.0,1.0,0.000694,119.0,2.648276
2,historical,Ain,1,1984,282.05420,6.998279,286.64294,8.002853,0.027781,0.000125,6.0,1.0,0.000987,109.0,4.822581
3,historical,Ain,1,1985,281.67966,7.407989,286.12997,8.244929,0.034206,0.000145,7.0,1.0,0.000887,125.0,4.196774
4,historical,Ain,1,1986,282.42148,6.688269,287.31296,7.404187,0.023955,0.000116,14.0,1.0,0.000724,93.0,3.598450


In [6]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 6141 entries, 0 to 6140
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   scenario              6141 non-null   str    
 1   department            6141 non-null   str    
 2   code_dep              6141 non-null   int64  
 3   year                  6141 non-null   int64  
 4   mean_temp             6141 non-null   float64
 5   std_temp              6141 non-null   float64
 6   mean_max_temp         6141 non-null   float64
 7   std_max_temp          6141 non-null   float64
 8   total_precip          6141 non-null   float64
 9   std_precip            6141 non-null   float64
 10  max_consec_rain_days  6141 non-null   float64
 11  min_consec_rain_days  6141 non-null   float64
 12  max_daily_precip      6141 non-null   float64
 13  rainy_days_count      6141 non-null   float64
 14  yield                 3293 non-null   float64
dtypes: float64(11), int64(2), str(2)

In [7]:
df.tail()

,scenario,department,code_dep,year,mean_temp,std_temp,mean_max_temp,std_max_temp,total_precip,std_precip,max_consec_rain_days,min_consec_rain_days,max_daily_precip,rainy_days_count,yield
6136,ssp1_2_6,Yvelines,78,2046,284.76740,8.022249,288.94733,9.058152,0.008247,0.000044,2.0,1.0,0.000349,29.0,NaN
6137,ssp1_2_6,Yvelines,78,2047,284.41516,6.287615,288.27660,7.179433,0.009969,0.000045,3.0,1.0,0.000298,37.0,NaN
6138,ssp1_2_6,Yvelines,78,2048,286.08868,7.381526,290.42615,8.591352,0.008904,0.000040,2.0,1.0,0.000319,30.0,NaN
6139,ssp1_2_6,Yvelines,78,2049,285.68747,8.106053,290.24106,9.455622,0.008438,0.000045,6.0,1.0,0.000287,33.0,NaN
6140,ssp1_2_6,Yvelines,78,2050,285.41495,6.969815,289.36765,7.847225,0.009245,0.000047,2.0,1.0,0.000349,33.0,NaN


In [8]:
# --- FEATURE ENGINEERING BLOCK ---
# RUN THIS BEFORE SPLITTING THE DATA INTO TRAIN/VAL

# 1. Sort Data to ensure 2014 (Historical) flows into 2015 (SSP) correctly
# This is crucial so that 2015 can 'look back' at 2014 for its lag features
df = df.sort_values(by=["department", "year"])

# 2. Define ALL weather columns you want to create lags for
# I have expanded this list to include heatwaves and extreme rain events
weather_cols = [
    "mean_temp", 
    "mean_max_temp",       # Captures heatwaves
    "total_precip", 
    "max_daily_precip",    # Captures storms/flooding events
    "rainy_days_count", 
    "max_consec_rain_days" # Captures prolonged wet periods
]

# 3. Create Features
for col in weather_cols:
    # A. 1-Year Lag (Immediate effect from previous year)
    # The .groupby() ensures we don't mix up data between different departments
    df[f"{col}_lag1"] = df.groupby("department")[col].shift(1)
    
    # B. 3-Year Rolling Trend
    # Captures longer-term climate shifts (e.g. "is it getting drier?")
    # .shift(1) ensures we don't use the current year's data to predict itself (leakage)
    df[f"{col}_trend_3y"] = df.groupby("department")[col].shift(1).transform(lambda x: x.rolling(3).mean())

# 4. Drop ONLY the rows where features are missing (the first 3 years of history)
# We use 'subset' so we don't accidentally drop future rows (2019+) where 'yield' is NaN
new_features = [f"{col}_lag1" for col in weather_cols] + [f"{col}_trend_3y" for col in weather_cols]
df = df.dropna(subset=new_features).reset_index(drop=True)

# Check the new columns
print(f"Added {len(new_features)} new features.")
print(df[["department", "year", "total_precip", "total_precip_lag1"]].head())

Added 12 new features.
  department  year  total_precip  total_precip_lag1
0        Ain  1985      0.034206           0.027781
1        Ain  1986      0.023955           0.034206
2        Ain  1987      0.035535           0.023955
3        Ain  1988      0.028758           0.035535
4        Ain  1989      0.039869           0.028758


In [9]:
df_train = df[df["year"]<2005]
df_val = df[(df["year"]<2019) & (df["year"]>=2005)]
df_test = df[df["year"]>=2019]

In [10]:
df_val.info()

<class 'pandas.DataFrame'>
Index: 1246 entries, 20 to 5841
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   scenario                       1246 non-null   str    
 1   department                     1246 non-null   str    
 2   code_dep                       1246 non-null   int64  
 3   year                           1246 non-null   int64  
 4   mean_temp                      1246 non-null   float64
 5   std_temp                       1246 non-null   float64
 6   mean_max_temp                  1246 non-null   float64
 7   std_max_temp                   1246 non-null   float64
 8   total_precip                   1246 non-null   float64
 9   std_precip                     1246 non-null   float64
 10  max_consec_rain_days           1246 non-null   float64
 11  min_consec_rain_days           1246 non-null   float64
 12  max_daily_precip               1246 non-null   float64
 13  rai

In [11]:
meta_val = df_val[["year", "department"]].reset_index(drop=True)

X_train = df_train.drop(columns = ["scenario", "year","yield"])
X_val = df_val.drop(columns = ["scenario", "year","yield"])
X_test = df_test.drop(columns = ["scenario", "year","yield"])
y_train = df_train["yield"]
y_val = df_val["yield"]

In [12]:
X_train.head()

,department,code_dep,mean_temp,std_temp,mean_max_temp,std_max_temp,total_precip,std_precip,max_consec_rain_days,min_consec_rain_days,...,mean_max_temp_lag1,mean_max_temp_trend_3y,total_precip_lag1,total_precip_trend_3y,max_daily_precip_lag1,max_daily_precip_trend_3y,rainy_days_count_lag1,rainy_days_count_trend_3y,max_consec_rain_days_lag1,max_consec_rain_days_trend_3y
0,Ain,1,281.67966,7.407989,286.12997,8.244929,0.034206,0.000145,7.0,1.0,...,286.64294,286.786470,0.027781,0.028096,0.000987,0.000851,109.0,113.000000,6.0,7.666667
1,Ain,1,282.42148,6.688269,287.31296,7.404187,0.023955,0.000116,14.0,1.0,...,286.12997,286.335570,0.034206,0.030333,0.000887,0.000856,125.0,117.666667,7.0,6.666667
2,Ain,1,281.83755,7.087537,286.31256,7.909338,0.035535,0.000154,12.0,1.0,...,287.31296,286.695290,0.023955,0.028647,0.000724,0.000866,93.0,109.000000,14.0,9.000000
3,Ain,1,282.47012,7.818445,287.24512,8.926070,0.028758,0.000118,10.0,1.0,...,286.31256,286.585163,0.035535,0.031232,0.001018,0.000876,135.0,117.666667,12.0,11.000000
4,Ain,1,282.11350,6.946863,286.22690,7.801536,0.039869,0.000156,12.0,1.0,...,287.24512,286.956880,0.028758,0.029416,0.000763,0.000835,123.0,117.000000,10.0,12.000000


In [13]:
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_percentage_error

# --- 1. Define Preprocessor ---
# We keep this similar to your original code to maintain compatibility.
cat_col = ["department"]
#num_col = X_train.select_dtypes(include="number").columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        # handle_unknown='ignore' is a safe setting for validation/test data
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_col)
        #("num", StandardScaler(), num_col) 
    ],
    remainder="passthrough"
)

In [36]:
# --- 2. Define XGBoost Model ---
# We set n_jobs=-1 to use all processor cores for faster training.
model = XGBRegressor(
    n_estimators=500,    # Number of trees (higher is usually better with early stopping or high regularisation)
    learning_rate=0.2,   # Lower rate (0.05) generalizes better than default (0.3)
    max_depth=3,          # Depth of trees (controls complexity)
    n_jobs=-1,
    random_state=42
)

In [37]:
# --- 3. Create Pipeline & Train ---
pipe = make_pipeline(preprocessor, model)

print("Training XGBoost model...")
pipe.fit(X_train, y_train)

# --- 4. Evaluate on Validation Set ---
y_pred_val = pipe.predict(X_val)

r_2 = r2_score(y_val, y_pred_val)
mape = mean_absolute_percentage_error(y_val, y_pred_val)

print("-" * 30)
print(f"Validation R²:  {r_2:.4f}")
print(f"Validation MAPE: {mape:.4f}")
print("-" * 30)

Training XGBoost model...
------------------------------
Validation R²:  0.4454
Validation MAPE: 0.1394
------------------------------


In [16]:
pred_val_df = pd.DataFrame({
    "year": df_val["year"].values,
    "department": df_val["department"].values,
    "actual_yield": y_val.values,
    "predicted_yield": y_pred_val
})


Let's look at our prediction for a specific region

In [17]:
region_choice = "Ain"

In [18]:
region_perf = pred_val_df[
    pred_val_df["department"] == region_choice
].reset_index()

region_perf = (
    region_perf
    .groupby("year")
    .agg(
        actual=("actual_yield","mean"),
        predicted=("predicted_yield","mean")
    )
    .reset_index()
)



In [19]:

fig_perf = px.line(
    region_perf,
    x="year",
    y=["actual","predicted"],
    markers=True,
    title=f"Actual vs Predicted Yield in {region_choice}"
)

fig_perf.show()